### Analysis of Liverpool 2023 Voting
This analysis assumes different voting schemes, which all have in common that all countries apart from the last get points. This scheme would work for 25 participants, too, as then the last act would get 1 point instead of 0.

First case assumes that the points from 0 - 26 are distributed evenly, while the second scheme assumes that the second best gets 26 and the best 28 points.

The data is scraped from eurovision.tv for all countries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from rich import pretty

In [2]:
base_path = 'https://eurovision.tv/event/liverpool-2023/grand-final/results/{0}'
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
# countries = ('albania', 'armenia', 'australia', 'austria', 'belgium', 'croatia', 'cyprus', 'czechia', 'estonia', 'finland', 'france', 'germany', 'israel', 'italy', 'lithuania', 'moldova', 'norway', 'poland', 'portugal', 'serbia', 'slovenia', 'spain', 'sweden', 'switzerland', 'ukraine', 'united kingdom')
voting_type = ('juryA', 'juryB', 'juryC', 'juryD', 'juryE')
load_from_internet = True

Reading out the names of the voting countries from the list of countries entry

In [3]:

respones_for_countries = requests.get(base_path.format('albania'), headers=header)
countries_soup = BeautifulSoup(respones_for_countries.text, 'html.parser')


In [4]:
countries = []
country_selection = countries_soup.find('select', class_='form-select')
for con in country_selection.children:
    countries.append(con.text.lower().replace(' ', '-'))

First, lets create the final data structure to be populated. The data structure contains a line for each country, and the columns are the origins of the points.
The origins are named by the following scheme: '<country>_tele', '<country>_juryA', etc

In [5]:
columns_jury = []
columns_jury_final = []
columns_tele = []
for voted_to_coun in countries:
    for vote in voting_type:
        columns_jury.append(f'{voted_to_coun}_{vote}')
    columns_tele.append(f'{voted_to_coun}_tele')
    columns_jury_final.append(f'{voted_to_coun}_jury')
        
votes_jury = pd.DataFrame(index=countries, columns=columns_jury)
votes_tele = pd.DataFrame(index=countries, columns=columns_tele)

In [6]:
if load_from_internet:
    for voted_to_coun in countries:
        voted_to_coun = voted_to_coun.replace(' ', '-')
        print(base_path.format(voted_to_coun))
        response = requests.get(base_path.format(voted_to_coun), headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')
        detailed_voting = soup.find('table').find('tbody')
        rows = detailed_voting.findAll('tr')
        for row in rows:
            cols = row.findAll('td')
            origin = cols[0].text.strip().lower().replace(' ', '-')
            # Add jury votes
            votes_jury.loc[origin, f'{voted_to_coun}_juryA'] = int(cols[2].text.strip())
            votes_jury.loc[origin, f'{voted_to_coun}_juryB'] = int(cols[3].text.strip())
            votes_jury.loc[origin, f'{voted_to_coun}_juryC'] = int(cols[4].text.strip())
            votes_jury.loc[origin, f'{voted_to_coun}_juryD'] = int(cols[5].text.strip())
            if len(cols) == 9:
                votes_jury.loc[origin, f'{voted_to_coun}_juryE'] = int(cols[6].text.strip())
            # Add televotes (more complicated due to structure)
            votes_tele.loc[origin, f'{voted_to_coun}_tele'] = int(cols[-1].text.split(' ')[-1].strip('stndrh'))
else:
    votes_jury = pd.read_csv(r'voting_results_table_jury.csv')
    votes_tele = pd.read_csv(r'voting_results_table_tele.csv')

https://eurovision.tv/event/liverpool-2023/grand-final/results/albania
https://eurovision.tv/event/liverpool-2023/grand-final/results/armenia
https://eurovision.tv/event/liverpool-2023/grand-final/results/australia
https://eurovision.tv/event/liverpool-2023/grand-final/results/austria
https://eurovision.tv/event/liverpool-2023/grand-final/results/azerbaijan
https://eurovision.tv/event/liverpool-2023/grand-final/results/belgium
https://eurovision.tv/event/liverpool-2023/grand-final/results/croatia
https://eurovision.tv/event/liverpool-2023/grand-final/results/cyprus
https://eurovision.tv/event/liverpool-2023/grand-final/results/czechia
https://eurovision.tv/event/liverpool-2023/grand-final/results/denmark
https://eurovision.tv/event/liverpool-2023/grand-final/results/estonia
https://eurovision.tv/event/liverpool-2023/grand-final/results/finland
https://eurovision.tv/event/liverpool-2023/grand-final/results/france
https://eurovision.tv/event/liverpool-2023/grand-final/results/georgia
htt

In [7]:
# save data frame to file to not depend on internet connection
votes_jury.to_csv(r'voting_results_table_jury.csv')
votes_tele.to_csv(r'voting_results_table_tele.csv')

### Implement voting schema

#### First trying to reproduce the official voting scheme

This is not a piece of cake, as the voting system is maximally intransparent... See e.g. https://thateurovisionsite.com/2023/04/12/eurovision-2023-voting-rules/

Each jury member ranks from 1-25, this is converted to points from 12 to 1 decreasing exponentially (see https://de.wikipedia.org/wiki/Eurovision_Song_Contest#Neuregelung_ab_2018). These points from all judges are combined and then sorted again, and the first 10 get the usual points.

First, create exponential points from 12 to 1. The function for exponential decay without offset is

$y = A * exp(K * x)$

In [8]:
def exp_decay(x, A, K):
    return A * np.exp(K * x)

Then use scipy.optimize.curve_fit to find the exponential parameters to make the function pass through 12 and 1

In [14]:
params, cov = curve_fit(exp_decay, [1, 25], [12, 1])

c:\Users\Jakob\.virtualenvs\tests-iEd5lnFy\lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Calculate the jury points based on the found exponential function and make sure the boundary conditions fit

In [17]:
x = np.arange(1, 26)
jury_points = exp_decay(x, *params)

assert np.abs(jury_points[0] - 12.0) < 1e-6
assert np.abs(jury_points[-1] - 1.0) < 1e-6

jury_points = np.append(jury_points, 0)

Create ESC ranking points from 12 to 1 for the first 10 places

In [30]:
esc_points = np.zeros((27,))
esc_points[0:10] = np.array([12, 10, 8, 7, 6, 5, 4, 3, 2, 1])
ranking = np.array([i + 1 for i in range(27)])

Replace the jury ranking by the jury points

In [32]:
votes_jury_points_individual = votes_jury.replace(ranking[:-1], jury_points)

Sum all jury points from each country for each country together

In [33]:
votes_jury_combined = pd.DataFrame(index=countries, columns=columns_jury_final)

In [34]:
for voting_coun in countries:
    for voted_to_coun in countries:
        votes_jury_combined.loc[voting_coun, f'{voted_to_coun}_jury'] = votes_jury_points_individual.loc[voting_coun, (col for col in votes_jury if col.startswith(f'{voted_to_coun}'))].sum()

Convert the placing into a placing from 1st to last and replace with esc points from 12 to 0

In [35]:
votes_jury_ranking = pd.DataFrame(0, index=countries, columns=columns_jury_final)

for voting_coun in countries:
    votes_jury_ranking.loc[:, f'{voting_coun}_jury'] = votes_jury_combined.loc[:, f'{voting_coun}_jury'].rank(ascending=False, method='min').replace(ranking, esc_points)

Add all jury points for each country together

In [38]:
jury_votes_total = votes_jury_ranking.sum(axis=1)

Print the results for the jury votes calculated to my best knowledge as stated by the EBU

In [40]:
pretty.pprint(jury_votes_total.sort_values(ascending=False))

sweden            338.0
israel            178.0
italy             178.0
estonia           143.0
finland           137.0
belgium           127.0
australia         123.0
austria           114.0
czechia            94.0
spain              90.0
lithuania          85.0
armenia            76.0
cyprus             66.0
switzerland        60.0
norway             58.0
ukraine            55.0
france             51.0
portugal           41.0
slovenia           33.0
moldova            24.0
albania            19.0
united-kingdom     19.0
serbia             12.0
poland             11.0
croatia            11.0
germany             3.0
latvia              0.0
malta               0.0
ireland             0.0
netherlands         0.0
iceland             0.0
greece              0.0
romania             0.0
san-marino          0.0
georgia             0.0
denmark             0.0
azerbaijan          0.0
dtype: float64

Conclusion: There are some differences, but I am not sure, why. Checked that all steps do what they are supposed to do. I did not check yet that all jury votes for all countries match the real jury votes (i.e. who gets how many points), apart from Albania, where I noticed that they switched Estonia and Armenia in places...

Continuing with the televote

In [41]:
tele_votes_total = votes_tele.replace(ranking, esc_points).sum(axis=1)

In [42]:
pretty.pprint(tele_votes_total.sort_values(ascending=False))

finland           364.0
sweden            233.0
norway            208.0
ukraine           187.0
israel            185.0
italy             173.0
croatia           107.0
moldova            76.0
poland             74.0
albania            59.0
cyprus             58.0
armenia            53.0
belgium            49.0
france             46.0
lithuania          46.0
slovenia           45.0
czechia            35.0
switzerland        31.0
estonia            22.0
australia          18.0
austria            16.0
serbia             16.0
portugal           16.0
germany            15.0
united-kingdom      9.0
spain               5.0
san-marino          0.0
georgia             0.0
ireland             0.0
latvia              0.0
azerbaijan          0.0
greece              0.0
netherlands         0.0
denmark             0.0
malta               0.0
iceland             0.0
romania             0.0
dtype: float64

Televotes seem to match quite well with the real televotes, so the calculation seems to be ok

In [43]:
total_votes_as_in_show = jury_votes_total + tele_votes_total

In [45]:
pretty.pprint(total_votes_as_in_show.sort_values(ascending=False))

sweden            571.0
finland           501.0
israel            363.0
italy             351.0
norway            266.0
ukraine           242.0
belgium           176.0
estonia           165.0
australia         141.0
lithuania         131.0
austria           130.0
czechia           129.0
armenia           129.0
cyprus            124.0
croatia           118.0
moldova           100.0
france             97.0
spain              95.0
switzerland        91.0
poland             85.0
slovenia           78.0
albania            78.0
portugal           57.0
serbia             28.0
united-kingdom     28.0
germany            18.0
malta               0.0
latvia              0.0
netherlands         0.0
ireland             0.0
iceland             0.0
romania             0.0
san-marino          0.0
greece              0.0
georgia             0.0
denmark             0.0
azerbaijan          0.0
dtype: float64

Conclusion: Overall it looks ok, however, there are slight differences I can currently not explain.

#### Voting scheme: Everybody gets points from 26 to 1

This (and the following) voting scheme assumes that the ranking is calculated exactly the same for the jurys as it is now, and only the way points are awarded change.

For this scheme, I assume that the points from 26 to 1 are evenly distributed according to the ranking

In [49]:
equal_points = np.arange(25, -1, -1)
equal_points = np.append(equal_points, 0)

In [50]:
votes_jury_ranking_equal = pd.DataFrame(0, index=countries, columns=columns_jury_final)

for voting_coun in countries:
    votes_jury_ranking_equal.loc[:, f'{voting_coun}_jury'] = votes_jury_combined.loc[:, f'{voting_coun}_jury'].rank(ascending=False, method='min').replace(ranking, equal_points)
    
jury_votes_total_equal = votes_jury_ranking_equal.sum(axis=1)
tele_votes_total_equal = votes_tele.replace(ranking, equal_points).sum(axis=1)

In [55]:
pretty.pprint(jury_votes_total_equal.sort_values(ascending=False))

sweden            839.0
italy             682.0
israel            661.0
estonia           629.0
australia         583.0
belgium           581.0
austria           569.0
spain             558.0
finland           543.0
cyprus            538.0
armenia           521.0
czechia           508.0
lithuania         490.0
switzerland       489.0
france            464.0
norway            452.0
ukraine           445.0
portugal          376.0
slovenia          323.0
united-kingdom    320.0
poland            270.0
moldova           266.0
albania           265.0
serbia            253.0
croatia           203.0
germany           197.0
azerbaijan          0.0
denmark             0.0
georgia             0.0
latvia              0.0
san-marino          0.0
ireland             0.0
greece              0.0
iceland             0.0
netherlands         0.0
malta               0.0
romania             0.0
dtype: float64

In [56]:
pretty.pprint(tele_votes_total_equal.sort_values(ascending=False))

finland           861.0
sweden            764.0
norway            740.0
israel            691.0
ukraine           683.0
italy             679.0
croatia           564.0
moldova           554.0
france            533.0
poland            532.0
cyprus            464.0
czechia           459.0
slovenia          430.0
switzerland       419.0
belgium           413.0
germany           375.0
austria           363.0
armenia           352.0
lithuania         342.0
albania           334.0
australia         327.0
estonia           285.0
serbia            260.0
spain             208.0
portugal          197.0
united-kingdom    196.0
san-marino          0.0
azerbaijan          0.0
denmark             0.0
latvia              0.0
ireland             0.0
georgia             0.0
greece              0.0
netherlands         0.0
iceland             0.0
malta               0.0
romania             0.0
dtype: float64

In [51]:
total_votes_equal = tele_votes_total_equal + jury_votes_total_equal

In [54]:
pretty.pprint(total_votes_equal.sort_values(ascending=False))

sweden            1603.0
finland           1404.0
italy             1361.0
israel            1352.0
norway            1192.0
ukraine           1128.0
cyprus            1002.0
france             997.0
belgium            994.0
czechia            967.0
austria            932.0
estonia            914.0
australia          910.0
switzerland        908.0
armenia            873.0
lithuania          832.0
moldova            820.0
poland             802.0
croatia            767.0
spain              766.0
slovenia           753.0
albania            599.0
portugal           573.0
germany            572.0
united-kingdom     516.0
serbia             513.0
malta                0.0
latvia               0.0
netherlands          0.0
ireland              0.0
iceland              0.0
romania              0.0
san-marino           0.0
greece               0.0
georgia              0.0
denmark              0.0
azerbaijan           0.0
dtype: float64

In this voting scheme, Germany would have been 15th in the televotes, still last in jury votes, and in total in place 24

#### Points from 26 to 1, but jury only 30%

In this voting scheme, the jury vote accounts only for 30% of the total votes, while televotes make up 70%

In [63]:
total_votes_equal_less_jury = 7 * tele_votes_total_equal + 3 * jury_votes_total_equal

In [65]:
pretty.pprint(total_votes_equal_less_jury.sort_values(ascending=False))

sweden            7865.0
finland           7656.0
israel            6820.0
italy             6799.0
norway            6536.0
ukraine           6116.0
france            5123.0
cyprus            4862.0
czechia           4737.0
moldova           4676.0
belgium           4634.0
croatia           4557.0
poland            4534.0
switzerland       4400.0
austria           4248.0
australia         4038.0
armenia           4027.0
slovenia          3979.0
estonia           3882.0
lithuania         3864.0
germany           3216.0
albania           3133.0
spain             3130.0
serbia            2579.0
portugal          2507.0
united-kingdom    2332.0
malta                0.0
latvia               0.0
netherlands          0.0
ireland              0.0
iceland              0.0
romania              0.0
san-marino           0.0
greece               0.0
georgia              0.0
denmark              0.0
azerbaijan           0.0
dtype: float64

In this voting scheme, Schweden would still have won, finland second, germany would be 21nd.

#### Points distributed from 12 to 1, but to all

In this voting scheme, the points are distributed the same as for the jury vote, i.e. exponentially descending from 12 to 1

In [58]:
votes_jury_ranking_expon = pd.DataFrame(0, index=countries, columns=columns_jury_final)

jury_points_expanded = np.append(jury_points, 0.0)

for voting_coun in countries:
    votes_jury_ranking_expon.loc[:, f'{voting_coun}_jury'] = votes_jury_combined.loc[:, f'{voting_coun}_jury'].rank(ascending=False, method='min').replace(ranking, jury_points_expanded)
    
jury_votes_total_expon = votes_jury_ranking_expon.sum(axis=1)
tele_votes_total_expon = votes_tele.replace(ranking, jury_points_expanded).sum(axis=1)

In [59]:
pretty.pprint(jury_votes_total_expon.sort_values(ascending=False))

sweden            369.951676
italy             257.079089
israel            252.602724
estonia           230.380964
belgium           208.478007
finland           205.955926
australia         205.600833
austria           201.725472
spain             185.888819
czechia           174.847367
armenia           169.520467
cyprus            169.094683
lithuania         167.517879
switzerland       155.295331
france            145.830331
norway            145.277219
ukraine           141.696266
portugal          118.478418
slovenia          104.040538
united-kingdom     96.409590
moldova            88.507150
albania            86.618032
poland             82.575507
serbia             81.901620
croatia            66.909804
germany            62.759695
azerbaijan          0.000000
denmark             0.000000
georgia             0.000000
latvia              0.000000
san-marino          0.000000
ireland             0.000000
greece              0.000000
iceland             0.000000
netherlands         0.000000
malta               0.000000
romania             0.000000
dtype: float64

In [60]:
pretty.pprint(tele_votes_total_expon.sort_values(ascending=False))

finland           389.714512
sweden            302.015992
norway            281.929647
ukraine           262.059836
israel            259.918748
italy             252.357707
croatia           193.819298
moldova           180.075622
poland            173.856392
france            161.042234
cyprus            149.080642
czechia           135.535927
slovenia          134.393167
belgium           132.043445
switzerland       124.299800
armenia           122.486788
albania           121.060729
lithuania         113.811842
germany           105.100678
austria           104.232369
australia          99.875076
estonia            89.974811
serbia             82.568168
portugal           69.278871
united-kingdom     68.595313
spain              65.815792
georgia             0.000000
netherlands         0.000000
iceland             0.000000
romania             0.000000
san-marino          0.000000
malta               0.000000
denmark             0.000000
azerbaijan          0.000000
latvia              0.000000
greece              0.000000
ireland             0.000000
dtype: float64

In [61]:
total_votes_expon = tele_votes_total_expon + jury_votes_total_expon

In [62]:
pretty.pprint(total_votes_expon.sort_values(ascending=False))

sweden            671.967668
finland           595.670438
israel            512.521472
italy             509.436796
norway            427.206865
ukraine           403.756101
belgium           340.521452
estonia           320.355776
cyprus            318.175324
czechia           310.383294
france            306.872565
austria           305.957841
australia         305.475908
armenia           292.007255
lithuania         281.329721
switzerland       279.595131
moldova           268.582772
croatia           260.729102
poland            256.431899
spain             251.704611
slovenia          238.433705
albania           207.678761
portugal          187.757289
germany           167.860373
united-kingdom    165.004902
serbia            164.469788
malta               0.000000
latvia              0.000000
netherlands         0.000000
ireland             0.000000
iceland             0.000000
romania             0.000000
san-marino          0.000000
greece              0.000000
georgia             0.000000
denmark             0.000000
azerbaijan          0.000000
dtype: float64

In this scheme Germany would have been 19th in televote, last in jury vote, and 24th overall again.